In [1]:
! pip install tensorflow mealpy scikit-learn matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 564.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 1.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 1.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 1.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 529.2 kB/s eta 0:00:00a 0:00:01


In [ ]:
import os
import re
import time
import gc
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# ——— GPU & XLA CONFIG —————————————————————————————————————
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=0'
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Using GPU: {[gpu.name for gpu in gpus]}")
    except RuntimeError as e:
        print(f"GPU setup error: {e}")
else:
    print("No GPU found. Using CPU instead.")

def clear_memory():
    K.clear_session()
    tf.keras.backend.clear_session()
    gc.collect()

# ——— CHECKPOINT UTIL ————————————————————————————————————
def get_latest_checkpoint(dir_path):
    files = [f for f in os.listdir(dir_path) if f.startswith("model_checkpoint_amla_opta_epoch_")]
    print("Checkpoints found:", files)
    if not files:
        return None, 0
    epochs = [int(re.search(r'epoch_(\d+)', f).group(1)) for f in files if re.search(r'epoch_(\d+)', f)]
    idx = max(range(len(epochs)), key=lambda i: epochs[i])
    return os.path.join(dir_path, files[idx]), epochs[idx]

# ——— USER SETTINGS —————————————————————————————————————
selected_optimizer = 'adam'
train_dir      = './amla_images/train'
validation_dir = './amla_images/val'
test_dir       = './amla_images/test'
checkpoint_dir = './checkpoints_amla_opta'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, 'model_checkpoint_amla_opta_epoch_{epoch:02d}.keras')

# ——— DATA GENERATORS ————————————————————————————————————
batch_size = 2
datagen = ImageDataGenerator(rescale=1./255)
train_gen = datagen.flow_from_directory(train_dir, target_size=(224,224), batch_size=batch_size, class_mode='binary')
val_gen   = datagen.flow_from_directory(validation_dir, target_size=(224,224), batch_size=batch_size, class_mode='binary')
test_gen  = datagen.flow_from_directory(test_dir, target_size=(224,224), batch_size=batch_size, class_mode='binary', shuffle=False)

# ——— MODEL UTILITIES ——————————————————————————————————————
def freeze_all(model):
    for layer in model.layers:
        layer.trainable = False

def unfreeze_layers(model, mode):
    if mode == 'all':
        for layer in model.layers:
            layer.trainable = True
    elif mode == 'last5':
        for layer in model.layers[-5:]:
            layer.trainable = True

def build_model():
    inp = Input(shape=(224,224,3))
    vgg = VGG19(weights='imagenet', include_top=False, input_tensor=inp)
    dn  = DenseNet121(weights='imagenet', include_top=False, input_tensor=inp)
    freeze_all(vgg)
    freeze_all(dn)

    x1 = GlobalAveragePooling2D()(vgg.output)
    x2 = GlobalAveragePooling2D()(dn.output)
    x  = Concatenate()([x1, x2])
    x  = Dense(256, activation='relu')(x)
    out= Dense(1, activation='sigmoid', dtype='float32')(x)
    model = Model(inputs=inp, outputs=out)
    return model

# ——— EVALUATION FUNCTION FOR CSO ——————————————————————
def evaluate_lr(lr_val):
    lr = float(lr_val[0]) if isinstance(lr_val, (list, np.ndarray)) else float(lr_val)
    model = build_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])
    start = time.time()
    history = model.fit(train_gen, validation_data=val_gen, epochs=2, steps_per_epoch=3, validation_steps=2, verbose=0)
    val_acc = history.history['val_accuracy'][-1]
    duration = time.time() - start
    print(f"Evaluated lr={lr:.6f} → val_acc={val_acc:.4f} in {duration:.2f}s")
    clear_memory()
    return -val_acc

class CuckooSearch:
    def __init__(self, objective_function, n=5, pa=0.25, alpha=0.01, lower_bound=[1e-6], upper_bound=[1e-3], max_iter=10):
        self.f = objective_function
        self.n = n
        self.pa = pa
        self.alpha = alpha
        self.lb = np.array(lower_bound)
        self.ub = np.array(upper_bound)
        self.max_iter = max_iter
        self.d = len(lower_bound)

    def levy_flight(self):
        from scipy.special import gamma
        beta = 1.5
        sigma = (gamma(1 + beta) * np.sin(np.pi * beta / 2) / (gamma((1 + beta) / 2) * beta * 2**((beta - 1) / 2)))**(1 / beta)
        u = np.random.randn(self.d) * sigma
        v = np.random.randn(self.d)
        step = u / np.abs(v)**(1 / beta)
        return step

    def search(self):
        nests = np.random.uniform(self.lb, self.ub, size=(self.n, self.d))
        fitness = np.array([self.f(nest) for nest in nests])
        best_idx = np.argmin(fitness)
        best_nest = nests[best_idx].copy()
        best_fitness = fitness[best_idx]

        for t in range(self.max_iter):
            for i in range(self.n):
                step = self.alpha * self.levy_flight()
                new_nest = nests[i] + step * (nests[i] - best_nest)
                new_nest = np.clip(new_nest, self.lb, self.ub)
                new_fit = self.f(new_nest)
                if new_fit < fitness[i]:
                    nests[i] = new_nest
                    fitness[i] = new_fit
                    if new_fit < best_fitness:
                        best_nest = new_nest
                        best_fitness = new_fit

            for i in range(self.n):
                if random.random() < self.pa:
                    nests[i] = np.random.uniform(self.lb, self.ub, self.d)
                    fitness[i] = self.f(nests[i])
                    if fitness[i] < best_fitness:
                        best_nest = nests[i]
                        best_fitness = fitness[i]

            print(f"[CS Iter {t+1}/{self.max_iter}] Best fitness = {-best_fitness:.4f}")

        return best_nest, best_fitness

def run_cuckoo_search():
    print("🔍 Running Cuckoo Search to find best learning rate...")
    cs = CuckooSearch(evaluate_lr, n=5, pa=0.25, alpha=0.01, lower_bound=[1e-6], upper_bound=[1e-3], max_iter=5)
    best_nest, best_score = cs.search()
    best_lr = float(best_nest[0])
    print(f"✅ Best LR found: {best_lr:.6f} with validation accuracy {-best_score:.4f}")
    return best_lr

# ——— OPTIMIZER FACTORY ————————————————————————————————————
def get_opt(name, lr):
    if name == 'adamw':
        return tf.keras.optimizers.AdamW(learning_rate=lr, weight_decay=1e-5)
    elif name == 'nadam':
        return tf.keras.optimizers.Nadam(learning_rate=lr)
    else:
        return tf.keras.optimizers.Adam(learning_rate=lr)

# ——— LOAD FROM CHECKPOINT IF AVAILABLE ————————————————————
latest_ckpt, last_epoch = get_latest_checkpoint(checkpoint_dir)
if latest_ckpt and os.path.exists(latest_ckpt):
    print(f"Loading from: {latest_ckpt}")
    model = load_model(latest_ckpt, compile=False)
    print(f"Resuming from epoch {last_epoch}")
else:
    model = build_model()
    last_epoch = 0
    print("Starting fresh training")

ckpt_cb = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, verbose=1)

# ——— TRAINING PHASES ——————————————————————————————————————
lr_phase_0 = run_cuckoo_search()

phases = [
    (lr_phase_0, 10, []),
    (1e-5, 5, ['last5']),
    (1e-6, 5, ['all']),
]

histories = []

for idx, (lr, epochs, flags) in enumerate(phases):
    if last_epoch >= sum(p[1] for p in phases[:idx+1]):
        continue

    if 'all' in flags:
        unfreeze_layers(model, 'all')
    elif 'last5' in flags:
        unfreeze_layers(model, 'last5')

    model.compile(optimizer=get_opt(selected_optimizer, lr), loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(train_gen, validation_data=val_gen, epochs=last_epoch + epochs, initial_epoch=last_epoch,
                        callbacks=[ckpt_cb], steps_per_epoch=train_gen.samples // batch_size,
                        validation_steps=val_gen.samples // batch_size)
    histories.append(history)
    last_epoch += epochs

    if idx < len(phases) - 1:
        latest_ckpt, _ = get_latest_checkpoint(checkpoint_dir)
        clear_memory()
        model = load_model(latest_ckpt, compile=False)
        print(f"Cleared session and reloaded model from {latest_ckpt}")

# ——— PLOT HISTORY ————————————————————————————————————————
plt.figure()
for i, h in enumerate(histories):
    plt.plot(h.history['accuracy'], label=f'train_acc_phase{i}')
    plt.plot(h.history['val_accuracy'], label=f'val_acc_phase{i}')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

plt.figure()
for i, h in enumerate(histories):
    plt.plot(h.history['loss'], label=f'train_loss_phase{i}')
    plt.plot(h.history['val_loss'], label=f'val_loss_phase{i}')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# ——— FINAL EVALUATION ——————————————————————————————————————
val_loss, val_acc = model.evaluate(val_gen)
test_loss, test_acc = model.evaluate(test_gen)
print(f"Validation → loss={val_loss:.4f}, acc={val_acc:.4f}")
print(f"Test       → loss={test_loss:.4f}, acc={test_acc:.4f}")

test_gen.reset()
probs = model.predict(test_gen, steps=test_gen.samples // batch_size + 1).ravel()
preds = (probs > 0.5).astype(int)
labels = test_gen.classes
print(classification_report(labels, preds))
cm = confusion_matrix(labels, preds)
print("Confusion Matrix:\n", cm)
fpr, tpr, _ = roc_curve(labels, probs)
roc_auc = auc(fpr, tpr)
print(f"ROC AUC = {roc_auc:.4f}")

plt.figure()
plt.plot(fpr, tpr)
plt.plot([0,1], [0,1], linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curve')
plt.show()

# ——— FINAL CLEANUP ————————————————————————————————————————
clear_memory()

Using GPU: ['/physical_device:GPU:0']
Found 11124 images belonging to 2 classes.
Found 2386 images belonging to 2 classes.
Found 2386 images belonging to 2 classes.
Checkpoints found: ['model_checkpoint_amla_opta_epoch_20.keras', 'model_checkpoint_amla_opta_epoch_07.keras', 'model_checkpoint_amla_opta_epoch_04.keras', 'model_checkpoint_amla_opta_epoch_02.keras', 'model_checkpoint_amla_opta_epoch_15.keras', 'model_checkpoint_amla_opta_epoch_01.keras', 'model_checkpoint_amla_opta_epoch_13.keras', 'model_checkpoint_amla_opta_epoch_10.keras', 'model_checkpoint_amla_opta_epoch_11.keras', 'model_checkpoint_amla_opta_epoch_09.keras', 'model_checkpoint_amla_opta_epoch_18.keras', 'model_checkpoint_amla_opta_epoch_19.keras', 'model_checkpoint_amla_opta_epoch_05.keras', 'model_checkpoint_amla_opta_epoch_06.keras', 'model_checkpoint_amla_opta_epoch_14.keras', 'model_checkpoint_amla_opta_epoch_16.keras', 'model_checkpoint_amla_opta_epoch_03.keras', 'model_checkpoint_amla_opta_epoch_08.keras', 'mode